In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
bert=pd.read_csv('/content/drive/MyDrive/Final_Predictions/bert-large_predictions.tsv', sep='\t', encoding='utf-8')

In [3]:
bert.head()

,tweet_id,preprocessed_tweet,Labels
0,111083,tamifairchild1 kingmoyer therightmelissa realc...,1
1,110702,deepbiden govt system created coronathey want ...,1
2,110204,canadian hc mckinnondavid blocked pointing can...,1
3,109871,im consultant icu lots covid19 patients great ...,1
4,108890,comparing covid19 vaccine smallpox vaccine lik...,1


In [4]:
bert.rename(columns = {'Labels':'Bert_Labels'}, inplace=True)

In [5]:
bert.head()

,tweet_id,preprocessed_tweet,Bert_Labels
0,111083,tamifairchild1 kingmoyer therightmelissa realc...,1
1,110702,deepbiden govt system created coronathey want ...,1
2,110204,canadian hc mckinnondavid blocked pointing can...,1
3,109871,im consultant icu lots covid19 patients great ...,1
4,108890,comparing covid19 vaccine smallpox vaccine lik...,1


In [7]:
bert = bert[['tweet_id','Bert_Labels']]

In [8]:
import pandas as pd
roberta=pd.read_csv('/content/drive/MyDrive/Final_roberta/roberta-large_predictions.tsv', sep='\t', encoding='utf-8')

In [9]:
roberta.rename(columns = {'Labels':'RoBerta_Labels'}, inplace=True)

In [10]:
roberta = roberta[['tweet_id','RoBerta_Labels']]

In [11]:
roberta.head()

,tweet_id,RoBerta_Labels
0,111083,1
1,110702,1
2,110204,1
3,109871,1
4,108890,1


In [12]:
import pandas as pd
xlnet=pd.read_csv('/content/drive/MyDrive/xlnet-base/xlnet-predictions', sep='\t', encoding='utf-8')

In [13]:
xlnet.rename(columns = {'Labels':'xlnet_Labels'}, inplace=True)

In [14]:
xlnet = xlnet[['tweet_id','xlnet_Labels']]

In [15]:
xlnet.head()

,tweet_id,xlnet_Labels
0,111083,1
1,110702,1
2,110204,1
3,109871,1
4,108890,1


In [16]:
# data = bert[['tweet_id','label','Bert_Labels']],roberta[['RoBerta_Labels']],xlnet[['xlnet_Labels']]

In [17]:
# data.head()

In [18]:
df = pd.merge(left=bert,right=roberta)

In [19]:
df.head()

,tweet_id,Bert_Labels,RoBerta_Labels
0,111083,1,1
1,110702,1,1
2,110204,1,1
3,109871,1,1
4,108890,1,1


In [20]:
data = pd.merge(left = df, right = xlnet)

In [21]:
data.head()

,tweet_id,Bert_Labels,RoBerta_Labels,xlnet_Labels
0,111083,1,1,1
1,110702,1,1,1
2,110204,1,1,1
3,109871,1,1,1
4,108890,1,1,1


In [62]:
ensemble = data[['tweet_id','Bert_Labels','RoBerta_Labels','xlnet_Labels']]

In [63]:
ensemble.head()

,tweet_id,Bert_Labels,RoBerta_Labels,xlnet_Labels
0,111083,1,1,1
1,110702,1,1,1
2,110204,1,1,1
3,109871,1,1,1
4,108890,1,1,1


In [64]:
ensemble['final_label'] = ensemble.mode(axis=1)

In [65]:
ensemble.head()

,tweet_id,Bert_Labels,RoBerta_Labels,xlnet_Labels,final_label
0,111083,1,1,1,1
1,110702,1,1,1,1
2,110204,1,1,1,1
3,109871,1,1,1,1
4,108890,1,1,1,1


In [66]:
ensemble.tail()

,tweet_id,Bert_Labels,RoBerta_Labels,xlnet_Labels,final_label
5917,110109,1,1,1,1
5918,112348,1,1,1,1
5919,109363,1,1,1,1
5920,201848,0,0,0,0
5921,109268,1,1,1,1


In [67]:
# from sklearn.metrics import classification_report
# print(classification_report(data.label,ensemble.final_label,digits=4))

In [68]:
ensemble.rename(columns = {'final_label':'label'}, inplace=True)

In [69]:
ensemble.head()

,tweet_id,Bert_Labels,RoBerta_Labels,xlnet_Labels,label
0,111083,1,1,1,1
1,110702,1,1,1,1
2,110204,1,1,1,1
3,109871,1,1,1,1
4,108890,1,1,1,1


In [71]:
# final = data[['xlnet_Labels','tweet_id']]

In [72]:
# final.head()

In [73]:
# final.tail()

In [74]:
# label_ = ensemble[['label']]

In [75]:
# label_.tail()

In [76]:
# df = pd.concat([final,label_])

In [77]:
# df.head()

In [78]:
# df.tail()

In [79]:
final_df = ensemble[['tweet_id','label']]

In [80]:
final_df.head()

,tweet_id,label
0,111083,1
1,110702,1
2,110204,1
3,109871,1
4,108890,1


In [81]:
final_df['label'] = final_df['label'].map({0: 'Self_reports', 1: 'Vaccine_chatter'})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
final_df.head()

,tweet_id,label
0,111083,Vaccine_chatter
1,110702,Vaccine_chatter
2,110204,Vaccine_chatter
3,109871,Vaccine_chatter
4,108890,Vaccine_chatter


In [83]:
final_df.tail()

,tweet_id,label
5917,110109,Vaccine_chatter
5918,112348,Vaccine_chatter
5919,109363,Vaccine_chatter
5920,201848,Self_reports
5921,109268,Vaccine_chatter


In [84]:
final_df.label.value_counts()

Vaccine_chatter    5438
Self_reports        484
Name: label, dtype: int64

In [85]:
final_df.to_csv('/content/drive/MyDrive/Final_Ensemble/ensemble_submission.tsv', sep='\t', index=False)